In [1]:
# -*- coding: utf-8 -*-
import re
import json
import zipfile
from pathlib import Path

import pandas as pd

# ==== 경로 설정 ====
TL_ZIP = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\TL.zip")  # TL.zip 실제 위치로 수정
FOOD_CSV = Path(r"C:\Users\EL19\OneDrive - (주)엘릭서\바탕 화면\음식\MS_testRepo\food_ingredients_integration1.csv")  # CSV 위치로 수정
OUT_CSV = TL_ZIP.with_name("image_food_label_join_fast.csv")  # 결과 파일 (이전이랑 이름만 다르게)

# ==== 1) 파일 이름에서 음식명 추출 함수 ====
def extract_food_name(file_name: str) -> str:
    """
    예: A_13_A13001_가자미구이_30_09.jpg  ->  '가자미구이'
    """
    stem = file_name.rsplit(".", 1)[0]  # 확장자 제거
    parts = stem.split("_")
    for p in parts:
        if re.search(r"[가-힣]", p):
            return p
    return ""

# ==== 2) 식약처 CSV 불러오기 ====
food_df = pd.read_csv(FOOD_CSV)
print(f"CSV 식품명 개수: {len(food_df)}")

# ==== 3) ZIP 안 JSON 한 번 훑어서, 메타데이터 + unique 음식명 수집 ====
json_meta_list = []   # 각 JSON에 대한 {json_path, image_file, food_from_filename}
unique_food_names = set()

with zipfile.ZipFile(TL_ZIP) as z:
    json_names = [name for name in z.namelist() if name.endswith(".json")]
    total_json = len(json_names)
    print(f"ZIP 안 JSON 개수: {total_json}")

    for idx, name in enumerate(json_names, start=1):
        data = json.loads(z.read(name).decode("utf-8"))
        image_info = data["data"]["image_info"]
        image_file = image_info["file_name"]

        food_from_file = extract_food_name(image_file)

        json_meta_list.append({
            "json_path": name,
            "image_file": image_file,
            "food_from_filename": food_from_file,
        })

        if food_from_file:  # 빈 문자열은 제외
            unique_food_names.add(food_from_file)

        if idx % 1000 == 0 or idx == total_json:
            print(f"[1/3] JSON 메타 파싱 {idx}/{total_json} 완료")

print(f"서로 다른 음식 이름 개수: {len(unique_food_names)}")

# ==== 4) 음식 이름별로 한 번만 매칭해서 캐싱 ====
match_cache = {}  # food_name -> 매칭된 DataFrame (혹은 빈 DataFrame)

food_name_list = list(unique_food_names)
total_food_names = len(food_name_list)

for i, food_name in enumerate(food_name_list, start=1):
    # 1차: 완전 일치
    matches = food_df[food_df["식품명"] == food_name]

    # 2차: 부분 포함
    if len(matches) == 0:
        matches = food_df[food_df["식품명"].str.contains(food_name, na=False, regex=False)]

    match_cache[food_name] = matches  # 비어있을 수도 있음

    if i % 50 == 0 or i == total_food_names:
        print(f"[2/3] 음식명 매칭 {i}/{total_food_names} 완료")

# ==== 5) JSON 메타 + 캐시된 매칭 결과를 합쳐 최종 rows 생성 ====
rows = []

for idx, meta in enumerate(json_meta_list, start=1):
    food_from_file = meta["food_from_filename"]

    if not food_from_file or food_from_file not in match_cache:
        # 음식 이름이 없거나(한글 없음 등), 캐시에 없는 경우 → not_found
        rows.append({
            "json_path": meta["json_path"],
            "image_file": meta["image_file"],
            "food_from_filename": food_from_file,
            "식품코드": "",
            "식품명(B열)": "",
            "match_type": "not_found",
        })
    else:
        matches = match_cache[food_from_file]

        if matches is None or len(matches) == 0:
            # 캐시에 있지만, 실제 매칭 결과 없음
            rows.append({
                "json_path": meta["json_path"],
                "image_file": meta["image_file"],
                "food_from_filename": food_from_file,
                "식품코드": "",
                "식품명(B열)": "",
                "match_type": "not_found",
            })
        else:
            # 매칭된 후보 모두 기록 (여러 개일 수 있음)
            for _, row in matches.iterrows():
                rows.append({
                    "json_path": meta["json_path"],
                    "image_file": meta["image_file"],
                    "food_from_filename": food_from_file,
                    "식품코드": row["식품코드"],
                    "식품명(B열)": row["식품명"],
                    "match_type": "exact" if row["식품명"] == food_from_file else "contains",
                })

    if idx % 1000 == 0 or idx == len(json_meta_list):
        print(f"[3/3] 최종 row 생성 {idx}/{len(json_meta_list)} 완료")

# ==== 6) 결과 저장 ====
out_df = pd.DataFrame(rows)
out_df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("완료! 결과 CSV:", OUT_CSV)


CSV 식품명 개수: 14584
ZIP 안 JSON 개수: 185553
[1/3] JSON 메타 파싱 1000/185553 완료
[1/3] JSON 메타 파싱 2000/185553 완료
[1/3] JSON 메타 파싱 3000/185553 완료
[1/3] JSON 메타 파싱 4000/185553 완료
[1/3] JSON 메타 파싱 5000/185553 완료
[1/3] JSON 메타 파싱 6000/185553 완료
[1/3] JSON 메타 파싱 7000/185553 완료
[1/3] JSON 메타 파싱 8000/185553 완료
[1/3] JSON 메타 파싱 9000/185553 완료
[1/3] JSON 메타 파싱 10000/185553 완료
[1/3] JSON 메타 파싱 11000/185553 완료
[1/3] JSON 메타 파싱 12000/185553 완료
[1/3] JSON 메타 파싱 13000/185553 완료
[1/3] JSON 메타 파싱 14000/185553 완료
[1/3] JSON 메타 파싱 15000/185553 완료
[1/3] JSON 메타 파싱 16000/185553 완료
[1/3] JSON 메타 파싱 17000/185553 완료
[1/3] JSON 메타 파싱 18000/185553 완료
[1/3] JSON 메타 파싱 19000/185553 완료
[1/3] JSON 메타 파싱 20000/185553 완료
[1/3] JSON 메타 파싱 21000/185553 완료
[1/3] JSON 메타 파싱 22000/185553 완료
[1/3] JSON 메타 파싱 23000/185553 완료
[1/3] JSON 메타 파싱 24000/185553 완료
[1/3] JSON 메타 파싱 25000/185553 완료
[1/3] JSON 메타 파싱 26000/185553 완료
[1/3] JSON 메타 파싱 27000/185553 완료
[1/3] JSON 메타 파싱 28000/185553 완료
[1/3] JSON 메타 파싱 29000/185553 완료
[1/3] JSON 메